In [47]:
import os
import pickle

# gensim LDA
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel

# Visualise
import warnings
# warnings.filterwarnings('ignore')
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

C:\Users\diede\Personal Files [Local]\Applied Data Science\Thesis - CITYNET\venv_citynet3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [48]:
from diedfunctions import *

In [ ]:
from tqdm.notebook import tqdm
import re
import spacy

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 1500000
nlp.disable_pipes('ner', 'parser')

### Import Lemmatised Paragraphs

In [38]:
import os
import pickle

word_list_location = "../../../../data/enwiki_city_pairs_lemmatised/lemmatised_paragraphs/"
city_pair_wordlists = []
city_pairs = []
for file in os.scandir(word_list_location):
    with open(file.path, 'rb') as fp:
        city_pair_wordlists.append(pickle.load(fp))
        city_pairs.append(file.name.split('__')[1])

In [43]:
for i, x in enumerate(city_pair_wordlists):
    print(len(x), city_pairs[i])

933 berlin_milan
7419 london_berlin
2320 london_madrid
2523 london_milan
1037 madrid_berlin
1130 madrid_milan
7662 paris_berlin
21193 paris_london
3294 paris_madrid
3264 paris_milan


In [45]:
#wordlists[0][:10] 
paragraphs = [paragraph for city_pair in city_pair_wordlists for paragraph in city_pair]
len(paragraphs)

50775

### Vectorize

In [49]:
%%time
dictionary, corpus = vectorize(paragraphs, filter_extremes=True, MIN_DF=0.05, MAX_DF=0.9)

CPU times: total: 3.12 s
Wall time: 3.15 s


### Train

In [51]:
%%time
N_TOPICS = 10
lda_model, coherence_score, dictionary, corpus = train_lda_model(paragraphs, dictionary, corpus, N_TOPICS=N_TOPICS)

CPU times: total: 7.88 s
Wall time: 2min 13s


# Coherence score

In [55]:
coherence_score

0.6615428576449918

### Visualisation

In [53]:
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model) # my_models[0]['lda_model__2']['model']
gensimvis.prepare(lda_conv, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.153594  0.015370       1        1  15.382016
9      0.084141 -0.146272       2        1  13.134513
2     -0.024415 -0.004284       3        1  12.617084
7     -0.013962 -0.094001       4        1  11.384676
6     -0.225623  0.024707       5        1   9.147340
5      0.110575 -0.248447       6        1   8.673702
1      0.120099  0.104428       7        1   8.538409
4     -0.287914 -0.032801       8        1   7.581092
8      0.129888  0.211473       9        1   7.033470
3     -0.046385  0.169826      10        1   6.507698, topic_info=           Term          Freq         Total Category  logprob  loglift
175       opera  10611.000000  10611.000000  Default  30.0000  30.0000
270         art  15738.000000  15738.000000  Default  29.0000  29.0000
189       study  11730.000000  11730.000000  Default  28.0000  28.0000
261        film   7056.000000   7056.000000  Default  27.0000  27.0000
429  exhibition   6071.000000   6071.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
268       title    856.571544   3037.018932  Topic10  -4.9500   1.4665
37         year   1369.655752  14962.517175  Topic10  -4.4807   0.3412
129       world   1080.911710   7132.423991  Topic10  -4.7174   0.8453
239      summer    715.956416   2140.107638  Topic10  -5.1294   1.6372
109      record    715.956416   3692.503351  Topic10  -5.1294   1.0917

[1036 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
885       4  0.978900         1490
646      10  0.931621     15706min
1229     10  0.931621       192324
1543      7  0.951371     19961997
1496      3  0.959189        71985
...     ...       ...          ...
1606      2  0.962336  zoroastrian
2196      1  0.959089          zsp
9711      6  0.995285        étude
1402      5  0.968182      čavajda
714       8  0.962798         żyli

[2654 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 10, 3, 8, 7, 6, 2, 5, 9, 4])

### Word Topic Distribution

In [52]:
MAX_WORDS = 8

for i in range(N_TOPICS):
    topic_words = lda_model.show_topic(i, topn=MAX_WORDS)
    print(i+1, [(x[0], round(x[1], 3)) for x in topic_words])

1 [('war', 0.012), ('german', 0.009), ('government', 0.007), ('british', 0.007), ('french', 0.007), ('foreign', 0.005), ('time', 0.005), ('return', 0.004)]
2 [('company', 0.019), ('city', 0.018), ('office', 0.016), ('open', 0.011), ('include', 0.01), ('world', 0.009), ('international', 0.008), ('business', 0.007)]
3 [('film', 0.029), ('fashion', 0.012), ('work', 0.012), ('publish', 0.011), ('festival', 0.01), ('book', 0.01), ('write', 0.009), ('model', 0.009)]
4 [('win', 0.036), ('final', 0.019), ('club', 0.018), ('team', 0.017), ('season', 0.015), ('time', 0.013), ('game', 0.013), ('play', 0.011)]
5 [('opera', 0.074), ('role', 0.024), ('perform', 0.019), ('sing', 0.016), ('theatre', 0.016), ('performance', 0.014), ('production', 0.014), ('debut', 0.014)]
6 [('study', 0.042), ('school', 0.02), ('professor', 0.015), ('university', 0.015), ('work', 0.013), ('year', 0.012), ('member', 0.011), ('society', 0.01)]
7 [('music', 0.029), ('perform', 0.025), ('orchestra', 0.024), ('tour', 0.019)

### Document Topic Distribution

In [ ]:
import pandas as pd

transformed_docs = lda_model.load_document_topics()
topic_distributions = pd.DataFrame([[x[1] for x in doc] for doc in transformed_docs], 
             columns=['topic_{}'.format(i+1) for i in range(N_TOPICS)])
topic_distributions

In [ ]:
topic_distributions.idxmax(axis=1).value_counts()

In [ ]:
distribution = topic_distributions.mean().sort_values(ascending=False)
sorted_distribution = distribution.sort_index()
distribution

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = sorted_distribution.index
students = list(sorted_distribution)
ax.bar(langs,students)
average_topic_coverage = sum(sorted_distribution)/len(sorted_distribution)

plt.axhline(average_topic_coverage, color='grey', ls='-', label='average topic coverage')
plt.legend(loc='lower right')
plt.show()

### Save Model

In [ ]:
keyname= 'allparagraphsof10biggestcitypairs10topics0_05and0_9coherence0_61'
model_path = f'../../../../data/lda_models/{keyname}.model'
lda_model.save(model_path)

In [15]:
# !jupyter nbextension enable --py widgetsnbextension

In [7]:
#from tqdm import tnrange
from tqdm.notebook import tqdm
import time

In [25]:
for i in tqdm(range(8)):
    for j in tqdm(range(10), leave=False):
    #for j in tqdm(tnrange(100), leave=False):
        time.sleep(0.1)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [44]:
l = list(range(20))

In [24]:
# Trash

In [22]:
def myprogress(current, total, whole=1, n=20, bars=u'▕▏▎▍▌▋▊▉', full='▉', empty='▕'):
    progress = current/total
    """ current and whole can be an element of a list being iterated, or just two numbers """
    p = (whole.index(progress))/len(whole)+1e-9 if type(whole)==list else progress/whole+1e-9 
    # print(p)
    return f"{full*int(p*n)}{bars[int(len(bars)*((p*n)%1))]}{empty*int((1-p)*n)} {p*100:04.1f}% {current:2.0f}/{total:1.0f}" 

In [26]:
for x in range(10):  ## accepting numerical value
    #print(myprogress(x/310), ' '*5, myprogress(x/321), end='\r')
    for busyloop in range(1000):
        time.sleep(0.01)
        if busyloop != 0 and (busyloop/1000*100)%10 == 0:
            print(myprogress(x, total=10), ' '*5, myprogress(busyloop, total=1000), end='\r')